In [18]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # pytorch has a lot of datasets # https://docs.pytorch.org/vision/stable/datasets.html
import torchvision.transforms as transforms
import torchvision

In [19]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# Hyperparameters
in_channel = 1
num_classes = 10
learning_rate = 1e-3 # 0.001
batch_size = 1024
num_epochs = 10
load_model = True

In [21]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [22]:
# Load pretrain model & modify it
# https://docs.pytorch.org/vision/stable/models.html
# https://docs.pytorch.org/vision/stable/models/vgg.html

# model = torchvision.models.vgg16(pretrained=True)
model = torchvision.models.vgg16(weights='DEFAULT')

# Freeze the pre-trained layers so they do not get updated during backprop, then replace the classifier with a new trainable head for our 10-class task
for param in model.parameters():
    param.requires_grad = False

# print(model)
model.avgpool = Identity()
# model.classifier = nn.Linear(512, 10)
model.classifier = nn.Sequential(nn.Linear(512, 100),
                                nn.ReLU(),
                                nn.Linear(100, 10))
model.to(device)
# model.classifier[0] = nn.Linear(512, 10)
# model.classifier = nn.Sequential(nn.Linear(512, 100),
#                                  nn.Dropout(p=0.5)
#                                 nn.Linear(100, 10))

# for i in range(1, 7):
#     model.classifier[i] = Identity
# print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [23]:
# Load Data
train_dataset = datasets.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [24]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
# Train network

for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    mean_loss = sum(losses) / len(losses)
    print(f'Loss at epoch {epoch} was {mean_loss:.5f}')

Loss at epoch 0 was 1.59202
Loss at epoch 1 was 1.21465
Loss at epoch 2 was 1.14633
Loss at epoch 3 was 1.11154
Loss at epoch 4 was 1.08883
Loss at epoch 5 was 1.07301
Loss at epoch 6 was 1.06238
Loss at epoch 7 was 1.05055
Loss at epoch 8 was 1.03940
Loss at epoch 9 was 1.03060


In [ ]:
# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            scores.max()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)